<a href="https://colab.research.google.com/github/cbalkig/Anomaly_Detection_in_Videos/blob/master/train_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import listdir
from os.path import join, isdir
import numpy as np
from PIL import Image
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, TimeDistributed, Conv2D, Flatten, Dense, Dropout, Activation, MaxPooling2D, Activation
from keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Conv3D, BatchNormalization
import matplotlib.pyplot as plt
import time
from numpy import save, load
from tensorflow.keras.optimizers import Adam

In [ ]:
working_directory = '/content/drive/MyDrive/AnomalyDetectionInVideos'

In [ ]:
class Config:
    DATASET_PATH = os.path.join(working_directory, "files")
    BATCH_SIZE = 16
    EPOCHS = 10
    MODEL_PATH = os.path.join(working_directory, "model.hdf5")
    SAMPLES_COUNT = 0
    FILTERS_COUNT = 4
    IMAGE_SIZE = 256

In [ ]:
def get_samples_data():
    sample_count = 0
    # Calculate the min number of frames in a video
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            sample_count = sample_count + len(listdir(directory_path))
            for c in listdir(directory_path):
              if len(c) < 7:
                os.rename(join(directory_path, c), join(directory_path, c.zfill(7)))
    print("Number of samples", sample_count)
    Config.SAMPLES_COUNT = sample_count

In [ ]:
get_samples_data()

In [ ]:
def get_training_set():
    # Process all videos and generate videos np array with shape: (Config.SAMPLES_COUNT, Config.IMAGE_SIZE, Config.IMAGE_SIZE)
    data = []
    labels = []
    sample_count = 0
    max_count = 0
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            print("Processing", directory_path)
            count = 0
            for c in sorted(listdir(directory_path)):
                #print("Frame #", count)
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((Config.IMAGE_SIZE, Config.IMAGE_SIZE))
                    img = np.array(img, dtype=np.float32) / 256.0
                    img = np.reshape(img, img.shape + (1,))
                    data.append(img)
                    labels.append(count)
                    sample_count = sample_count + 1
                    if count > max_count:
                        max_count = count
                    count = count + 1
    return data, labels, max_count

In [ ]:
train_data, train_labels, max_count = get_training_set()
print("Number of classes", max_count)
train_data = np.array(train_data)
train_labels = np.array(train_labels)
save(join(Config.DATASET_PATH, 'train_data_' + str(Config.IMAGE_SIZE) + '.npy'), train_data)
print("Shape of training set", train_data.shape)
save(join(Config.DATASET_PATH, 'train_labels_' + str(Config.IMAGE_SIZE) + '.npy'), train_labels)
print("Shape of training set", train_labels.shape)

In [ ]:
training_set = load(join(Config.DATASET_PATH, 'train_data_' + str(Config.IMAGE_SIZE) + '.npy'))

In [ ]:
def make_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    model.summary()
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [ ]:
def train_model(reload_model=True):
    if reload_model:
        return load_model(Config.MODEL_PATH)
    model = make_model((Config.IMAGE_SIZE, Config.IMAGE_SIZE, 1), max_count + 1)
    history = model.fit(train_data, one_hot_train_labels, epochs=Config.EPOCHS, batch_size=Config.BATCH_SIZE)
    print(history)
    return model

In [ ]:
one_hot_train_labels = np.zeros((len(train_labels), max_count + 1))
for idx in range(len(train_labels)):
    one_hot_train_labels[idx][train_labels[idx]] = 1      

In [ ]:
model = train_model(reload_model=False)
print("Model is ready.")
model.save(Config.MODEL_PATH)
print("Model saved : ", Config.MODEL_PATH)